# Práctica de aprendizaje automático (parte 1)

# Teorema de Bayes y Naïve Bayes (1.5 puntos)


## 1. Teorema de Bayes

El **Teorema de Bayes**:
$$
P(H_1 \mid D, I) = \frac{P(D \mid H_1, I) \times P(H_1 \mid I)}{P(D \mid I)} \quad \text{con} \quad P(D \mid I) = \sum_i P(D \mid H_i, I) \times P(H_i \mid I).
$$

- **$H_1$** = Hipótesis 1, **$D$** = datos, **$I$** = información.

- **$P(H_1 \mid I)$**: Probabilidad a *priori* de la hipótesis 1 (antes de observar los datos).
- **$P(D \mid I)$**: Evidencia de los datos.
- **$P(D \mid H_1, I)$**: *Verosimilitud* de la hipótesis 1 dados los datos.
- **$P(H_1 \mid D, I)$**: Probabilidad a *posteriori* de la hipótesis 1 (después de observar los datos).


### Ejercicio 1 - Palabra sospechosa en correo
Vamos a calcular la probabilidad de que un correo sea **SPAM** dadas las palabras **"gratis"** y **"oferta"**. Supongamos que dado un nuevo correo, la probabilidad a priori de que sea **SPAM** es del **20%**.

Si aparece la palabra **"gratis"**:
- $P(\text{"gratis"}\mid\text{SPAM})=0.4$
- $P(\text{"gratis"}\mid \neg \text{SPAM})=0.05$
- 
Si aparece la palabra **"oferta"**:
- $P(\text{"oferta"}\mid\text{SPAM})=0.7$
- $P(\text{"oferta"}\mid \neg \text{SPAM})=0.05$

1. Calcula $P(\text{SPAM}\mid\text{"gratis"})$.
2. ¿Qué ocurre si además ves la palabra **"oferta"** y asumes independencia? (Pista: multiplica verosimilitudes.)


Completa y ejecuta la celda de abajo. Además, justifica razonadamente el resultado que obtengas. 


In [1]:
p_spam = 0.2
p_no_spam = 1 - p_spam

p_gratis_spam = 0.4
p_gratis_no_spam  = 0.05

p_oferta_spam = 0.7
p_oferta_no_spam  = 0.05

# Tu código aquí
# ...

print(f"P(SPAM | 'gratis') = {posterior_gratis:.6f}  (~{100*posterior_gratis:.2f}%)")
print(f"P(SPAM | 'gratis' y 'oferta') = {posterior_gratis_oferta:.6f}  (~{100*posterior_gratis_oferta:.2f}%)")

P(SPAM | 'gratis') = 0.666667  (~66.67%)
P(SPAM | 'gratis' y 'oferta') = 0.965517  (~96.55%)


## Ejercicio 2 - Probabilidad de jugar profesionalmente al baloncesto

Ahora vamos a calcular la probabilidad de que una persona juegue profesionalmente al baloncesto dada su altura y su edad. La *probabilidad a priori* de que una persona sea **PRO** es del **1%**: $P(\text{PRO})=0.01$. Asumiremos **independencia condicional** entre *edad* y *altura* dada la clase (al asumir independencia estamos calculando "Naïve Bayes"). Además, consideraremos que las características se modelan con distribuciones **gaussianas**.

Estas son las verosimilitudes:

- Si es **PRO**  
  $\text{edad}\sim \mathcal{N}(\mu{=}27,\ \sigma^2{=}3^2)$,  
  $\text{altura (m)}\sim \mathcal{N}(\mu{=}2.00,\ \sigma^2{=}0.07^2)$.
- Si es **NO\_PRO**  
  $\text{edad}\sim \mathcal{N}(\mu{=}35,\ \sigma^2{=}10^2)$,  
  $\text{altura (m)}\sim \mathcal{N}(\mu{=}1.72,\ \sigma^2{=}0.06^2)$.

Calcula $P(\text{PRO}\mid \text{edad},\text{altura})$ para:

1) Persona A: **edad = 25**, **altura = 1.95 m**.  
2) Persona B: **edad = 30**, **altura = 1.90 m**.

> **Pista:** como **asumimos independencia condicional**, la verosimilitud conjunta se obtiene **multiplicando** las densidades de cada característica.
> $\text{score}(\text{PRO}) = P(\text{PRO})\cdot \mathcal{N}(\text{edad})\cdot \mathcal{N}(\text{altura})$.  
> Posterior $=\dfrac{\text{score}(\text{PRO})}{\text{score}(\text{PRO}) + \text{score}(\text{NO\_PRO})}$.

Completa y ejecuta la celda de abajo. Además, justifica razonadamente el resultado que obtengas. 


In [2]:
import numpy as np

# Prior
p_pro = 0.01
p_no_pro = 1 - p_pro

# Parámetros (medias y varianzas)
mu_edad_pro, var_edad_pro = 27, 3**2
mu_alt_pro,  var_alt_pro  = 2.00, 0.07**2

mu_edad_no, var_edad_no = 35, 10**2
mu_alt_no,  var_alt_no  = 1.72, 0.06**2

# Candidatos
edad_A, alt_A = 25, 1.95
edad_B, alt_B = 30, 1.90

def gaussian_pdf(x, mu, var):
    return 1/np.sqrt(2*np.pi*var) * np.exp(-(x-mu)**2/(2*var))

# Tu código aquí
# ...

print(f"P(PRO | A) = {posterior_personaA:.6f} (~{100*posterior_personaA:.2f}%)")
print(f"P(PRO | B) = {posterior_personaB:.6f} (~{100*posterior_personaB:.2f}%)")

P(PRO | A) = 0.978639 (~97.86%)
P(PRO | B) = 0.391573 (~39.16%)


## 2. Clasificador **Gaussian Naïve Bayes**

Ahora vamos a implementar un clasificador basado en Naïve Bayes, que solo acepta características continuas. Para ello, la única librería que emplearemos será `numpy`. El clasificador **Gaussian Naïve Bayes** asume que las características son **independientes entre sí dada la clase** (por esa razón se llama "naïve") y modela cada característica con una **Gaussiana**.
> **Nota:** Existen variantes de Naïve Bayes que emplean otras distribuciones según el tipo de dato (p. ej., Multinomial, Bernoulli), y también para datos continuos pueden usarse alternativas a la gaussiana.

El conjunto de datos de entrada es $\mathcal{D}=\{\mathbf{X}, \mathbf{y}\}$, donde $\mathbf{X}=\{\mathbf{x}_1, ..., \mathbf{x}_N\}$, $\mathbf{y} = \{y_1,\dots,y_N\}$, $N$ es el número de datos, $\mathbf{x}_i=(x_{i,1},\dots,x_{i,D})$ y $D$ es el número de características.

### `fit(X, y)`
Método para ajustar el clasificador (realiza el entrenamiento). Primero estima los **priores** de clase $P(y{=}k)$ como la frecuencia de cada clase en el conjunto de entrenamiento. Después, para cada clase $k$ y característica $j$, calcula la **media** $\mu_{j,k}$ y la **varianza** $\sigma^2_{j,k}$ usando únicamente los datos de esa clase.

- **Prior (frecuencia relativa):**
  $$
  P(y{=}k)=\frac{n_k}{N}.
  $$
  (donde $n_k$ es el número de ejemplos de la clase $k$ y $N$ el total).

> **Nota:** añade un $\varepsilon$ pequeño a cada $\sigma^2_{j,k}$ para evitar varianzas nulas.

- **Salida:** el clasificador entrenado con las clases, los priors $\{P(y{=}k)\}$ y las matrices de medias $\mu$ y varianzas $\sigma^2$ de tamaño $(K\times D)$.

### `predict(X)`
Método de predicción. Para cada nuevo dato $x_i$, calcula la *probabilidad a posteriori* de cada clase $P(y{=}k \mid x_i)$ usando el prior y el producto de densidades Gaussianas por característica. Termina prediciendo la clase con mayor probabilidad.

- **Puntuación por clase:**
  $$
  \text{score}(k)= P(y{=}k)\, \prod_{j=1}^{D}\mathcal{N}\big(x_j;\,\mu_{j,k},\,\sigma^2_{j,k}\big).
  $$

- **Salida:** la predicción final es la clase que tenga mayor score (probabilidad).
  $$
  y=\arg\max_{k}\ \text{score}(k).
  $$


In [3]:
# Cargamos librerias
import numpy as np
from sklearn.datasets import load_iris

In [4]:
# 1) Cargar datos
X_all, y_all = load_iris(return_X_y=True)
np.random.seed(0)

# Dividimos el dataset entre train-test y validación (75-25)
indices = np.random.permutation(len(y_all))
cut = int(0.75 * len(y_all))
indices_train, indices_validation = indices[:cut], indices[cut:]
X_train, y_train = X_all[indices_train], y_all[indices_train]
X_validation, y_validation = X_all[indices_validation], y_all[indices_validation]

In [5]:
# Completa la clase GaussianNaiveBayes
class GaussianNaiveBayes:
    
    def __init__(self, eps=1e-8):
        self.eps = eps
        
    def fit(self, X, y):
        self.classes = # Tu código aquí
        counts       = # Tu código aquí
        self.priors  = # Tu código aquí
        self.mu      = # Tu código aquí
        self.var     = # Tu código aquí (recuerda añadir un pequeño epsilon)
        return self
        
    def predict(self, X):
        predictions = []
        # TODO: usa self.priors, self.mu, self.var para calcular 'scores' y predice la clase con mayor score para cada fila de X
        # Tu código aquí
        return np.asarray(predictions)

In [6]:
# Entrenar y validar
gnb = GaussianNaiveBayes().fit(X_train, y_train)
y_pred = gnb.predict(X_validation)
print("Precisión (validación en el 25% restante):", (y_pred == y_validation).mean())

Precisión (validación en el 25% restante): 0.9473684210526315
